In [2]:
import numpy as np 
from utility_funcs import get_train_labels_test
from scipy import stats

ModuleNotFoundError: No module named 'pandas'

In [ ]:

import tensorflow as tf
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))

In [42]:
train, labels, test = get_train_labels_test()

In [43]:
train.head()

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,0,47,0,-0.196291,0.112395,1.0,0.329204,-1.004660,-0.131638,-0.127505,0.368702,-0.1,-0.963873,-0.985069,0.531893,4.751492
1,0,47,1,-0.447450,0.134454,1.0,-0.658407,0.162495,0.340314,-0.209472,-0.867176,0.2,-0.301301,0.082733,-0.231481,0.454390
2,0,47,2,0.326893,-0.694328,1.0,0.330088,0.473678,1.280479,-0.094718,0.535878,1.4,1.002168,0.449221,-0.586420,-4.736147
3,0,47,3,0.523184,0.751050,1.0,0.976991,-0.563287,-0.720269,0.793260,0.951145,-0.3,-0.995665,-0.434290,1.344650,0.429241
4,0,47,4,0.272025,1.074580,1.0,-0.136283,0.398579,0.044877,0.560109,-0.541985,-0.9,1.055636,0.812631,0.123457,-0.223359


In [44]:
labels.head()

,sequence,state
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1


In [45]:
test.head()

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,25968,684,0,2.427357,19.639706,1.0,-1.466372,-1.289973,-4.207928,2.486339,-2.493893,8.0,-1.123555,-1.673048,10.980453,0.419011
1,25968,684,1,-4.950541,-21.747899,1.0,0.983186,-0.569053,1.845924,-3.887978,1.727481,-2.9,0.395231,-0.882233,-1.871399,-0.008525
2,25968,684,2,1.136012,-10.756303,1.0,1.016814,0.964157,2.454749,0.312386,1.154198,-5.6,1.114162,1.525273,-11.584362,0.139812
3,25968,684,3,0.806028,6.504202,1.0,-0.179646,0.969221,-1.035153,-0.457195,0.254962,-2.7,-0.588873,0.608761,-4.241770,-0.462916
4,25968,684,4,1.288253,5.552521,1.0,-0.493805,-1.036124,-1.126402,2.008197,-0.730534,0.0,0.899566,-1.259615,-0.472222,-0.121483


In [ ]:
train.describe().round(2)

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
count,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.0,1558080.00,1558080.00,1558080.00,1558080.00
mean,12983.50,331.63,29.50,0.00,-0.00,-0.22,-0.00,-0.00,-0.00,-0.00,-0.00,-0.0,0.00,0.00,-0.00,-0.01
std,7496.32,195.83,17.32,2.66,4.40,2.30,3.93,1.68,1.59,3.35,3.24,4.5,2.59,1.92,4.53,39.12
min,0.00,0.00,0.00,-375.06,-434.60,-31.66,-408.38,-23.63,-74.98,-470.50,-407.01,-536.1,-270.35,-43.41,-427.06,-612.55
25%,6491.75,161.75,14.75,-0.50,-0.48,-0.65,-0.49,-0.47,-0.48,-0.49,-0.50,-0.5,-0.52,-0.48,-0.48,-0.58
50%,12983.50,335.00,29.50,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.0,-0.00,-0.00,0.00,0.00
75%,19475.25,501.00,44.25,0.48,0.49,0.33,0.49,0.47,0.51,0.49,0.48,0.5,0.51,0.48,0.49,0.57
max,25967.00,671.00,59.00,335.82,449.59,1.67,436.65,24.87,77.92,442.50,331.25,630.1,367.98,41.87,448.02,630.51


### tabular feature computations

In [ ]:
sensor_cols = [col for col in train.columns if 'sensor' in col]
train[sensor_cols] = train[sensor_cols].replace(0, 0.0001) # prevent 0 div

train_seq_subj=train[['subject', 'sequence']].drop_duplicates().reset_index(drop=True)
train_seq_subj_lab = train_seq_subj.merge(labels,on='sequence')

train_group = train.drop(['step', 'subject'], axis=1)
train_group = train_group[train_group['sequence'] < 2000] # work on subset of data

In [ ]:
class TsFunctions:
       
    def _rms(self,ts, axis=0):
        return np.sqrt(np.mean(np.square(ts), axis=axis))

    def _mean_abs(self, ts, axis=0):
        return np.mean(np.abs(ts), axis=axis)

    def quant_10(self, ts):
        return np.quantile(ts, 0.1)

    def quant_25(self, ts):
        return np.quantile(ts, 0.25)

    def quant_50(self,ts):
        return np.quantile(ts, 0.5)

    def quant_75(self, ts):
        return np.quantile(ts, 0.75)

    def quant_99(self, ts):
        return np.quantile(ts, 0.99)

    def skewness(self,ts, axis=0):
        return stats.skew(ts, axis=axis)

    def kurtosis(self,ts, axis=0):
        return stats.kurtosis(ts, axis=axis)

    def crest_factor(self, ts, axis=0):
        return np.amax(ts, axis=axis) / self._rms(ts, axis=axis)

    def shape_factor(self, ts, axis=0,):
        return self._rms(ts, axis=axis) / self._mean_abs(ts, axis=axis)

    def impulse_factor(self, ts, axis=0):
        return np.max(ts, axis=axis) / self._mean_abs(ts, axis=axis)

    def margin_factor(self, ts, axis=0):
        return np.max(ts, axis=axis) / np.square(self._mean_abs(ts, axis=axis))

    def energy(self,ts, axis=0):
        return np.mean(np.square(ts), axis=axis)






In [ ]:

t= TsFunctions()
train_group = train_group.groupby('sequence').agg(['min', 'max','mean', 'std', t.quant_10, t.quant_25, t.quant_50, t.quant_75, t.quant_99, t.skewness, t.kurtosis, t.crest_factor, t.shape_factor, t.impulse_factor,t.margin_factor, t.energy])
train_group.columns = train_group.columns.map(''.join)
train_group.head()

,sensor_00min,sensor_00max,sensor_00mean,sensor_00std,sensor_00quant_10,sensor_00quant_25,sensor_00quant_50,sensor_00quant_75,sensor_00quant_99,sensor_00skewness,...,sensor_12quant_50,sensor_12quant_75,sensor_12quant_99,sensor_12skewness,sensor_12kurtosis,sensor_12crest_factor,sensor_12shape_factor,sensor_12impulse_factor,sensor_12margin_factor,sensor_12energy
sequence,,,,,,,,,,,,,,,,,,,,,
0,-5.634467,7.534003,0.041744,1.431660,-0.839722,-0.647798,-0.148764,0.540958,4.445394,1.450589,...,-0.447997,3.222187,6.613402,-4.128561,21.285187,0.810110,1.887338,1.528950,0.350228,67.886661
1,-6.657651,5.870943,-0.069294,1.593907,-1.415842,-0.494204,-0.031298,0.670015,3.864304,-0.666247,...,-0.009591,0.197038,143.834510,-0.216472,3.088862,2.897976,1.947372,5.643435,0.220254,2489.633224
2,-2.797527,3.402628,-0.001546,1.147526,-1.498377,-0.597952,0.058733,0.667117,2.839985,0.119600,...,-0.004902,0.173167,0.963887,0.074364,1.936414,3.119238,1.435278,4.476974,17.947168,0.128189
3,-3.643740,4.856260,0.029869,1.769705,-1.855873,-1.118624,-0.119784,1.018547,4.123091,0.298301,...,-0.012361,0.110294,0.588572,-0.180355,2.122750,2.475246,1.403231,3.473343,20.332185,0.057463
4,-5.331530,5.455951,0.059132,2.014779,-2.203941,-0.893161,0.005410,1.458849,4.320178,-0.210688,...,0.002344,0.024616,0.127677,0.384676,-0.001164,2.657414,1.251301,3.325226,81.316677,0.002618
